### Data Source : CSV

In [1]:
import pandas as pd
df = pd.read_excel(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\sample1.xlsx')

df.to_csv('sample1.csv')

In [2]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os
from langchain.embeddings.openai import OpenAIEmbeddings

In [3]:
from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
# Load the documents
loader = CSVLoader(file_path=r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\1 to Many.csv')

In [6]:
# Create an index using the loaded documents
index_creator = VectorstoreIndexCreator(embedding=embeddings)
docsearch = index_creator.from_loaders([loader])

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain\indexes\vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [7]:
# Create a question-answering chain using the index
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=openai.api_key)
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [8]:
# Pass a query to the chain
query = "Summarize the doument"
response = chain({"question": query})

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [9]:
print(type(response['result']))

<class 'str'>


### Document Source: PDF doc

In [2]:
import os
import PyPDF2
from PyPDF2 import PdfReader

# Importing necessary modules from langchain and langchain_openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.callbacks import get_openai_callback
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
pdfpath = r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\Vedanta Cairn_May 2024_PO No.7300175294 (1).PDF'

In [5]:
def chunk_processing(pdf):
    """
    Process a PDF file, extracting text and splitting it into chunks.
    """
    pdf_reader = PdfReader(pdf)
        
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
        
    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text=text)
    return chunks

In [6]:
def embeddings(chunks):
    """
    Create embeddings for text chunks using OpenAI.
    """
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
    # Create vector store using FAISS
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_store

In [9]:
def generation(VectorStore):
    """
    Generate responses based on prompts and embeddings.
    """
    retriever = VectorStore.as_retriever()
    
    # Define template for prompts
    template = """Respond to the prompt based on the following context: {context}
    Questions: {questions}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Initialize ChatOpenAI model
    model = ChatOpenAI(openai_api_key=openai.api_key)
    
    # Define processing chain
    chain = (
        {"context": retriever, "questions": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    
    # Prompt user for input
    query = input("Insert Prompt: ")
    
    # Invoke the processing chain with user input
    output = chain.invoke(query)
    return output

In [11]:
type(generation(embeddings(chunk_processing(pdfpath))))

str

### Document Source: Docx

In [21]:
#Convert the docx to pdf
from docx2pdf import convert

def convert_docx_to_pdf(input_path, output_path):
    try:
        # Convert DOCX to PDF
        convert(input_path, output_path)
        print(f"Conversion complete. PDF saved at {output_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
convert_docx_to_pdf(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\document_0.docx', 'document_0.pdf')

100%|██████████| 1/1 [00:03<00:00,  3.68s/it]

Conversion complete. PDF saved at document_0.pdf


In [13]:
df =pd.read_csv(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\1 to Many.csv')
df.to_excel('1 to Many.xlsx')

### Creating a standarad Pipeline

In [19]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os
from langchain.embeddings.openai import OpenAIEmbeddings
import PyPDF2
from PyPDF2 import PdfReader

# Importing necessary modules from langchain and langchain_openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.callbacks import get_openai_callback
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
#Convert the docx to pdf
from docx2pdf import convert

import pandas as pd

from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
class DataTalk:

    def __init__(self, filepath):
        self.filepath = filepath

    def source_type(self):
        file = ''
        folderpath = os.path.dirname(self.filepath)
        filename = os.path.basename(self.filepath)
        if filename.endswith('.xlsx'):
            csvname = filename.split('.')[0] + '.csv'
            file = os.path.join(folderpath, csvname)
            df = pd.read_excel(self.filepath)
            df.to_csv(file)
        elif filename.endswith('.docx'):
            pdfname = filename.split('.')[0] + '.pdf'
            file = os.path.join(folderpath, pdfname)
            convert(self.filepath, file)
        else: 
            file = self.filepath

        return file
    
    
    def generate_response(self, query):
        # Initialize the OpenAI embeddings
        embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
        model_name="gpt-3.5-turbo-instruct"
        file = self.source_type()
        response = ''

        if file.endswith('.csv'):
            # Load the documents
            loader = CSVLoader(file_path=file)
            # Create an index using the loaded documents
            index_creator = VectorstoreIndexCreator(embedding=embeddings)
            docsearch = index_creator.from_loaders([loader])
            # Create a question-answering chain using the index
            llm = OpenAI(model_name=model_name, openai_api_key=openai.api_key)
            chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
            response = chain({"question": query})
            response = response['result']
        
        elif file.lower().endswith('.pdf'):

            """
            Process a PDF file, extracting text and splitting it into chunks.
            """
            pdf_reader = PdfReader(file)
                
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
                
            # Split text into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            chunks = text_splitter.split_text(text=text)

            # Create vector store using FAISS
            vector_store = FAISS.from_texts(chunks, embedding=embeddings)

            """
            Generate responses based on prompts and embeddings.
            """
            retriever = vector_store.as_retriever()
            
            # Define template for prompts
            template = """Respond to the prompt based on the following context: {context}
            Questions: {questions}
            """
            prompt = ChatPromptTemplate.from_template(template)

            # Initialize ChatOpenAI model
            model = ChatOpenAI(openai_api_key=openai.api_key)
            
            # Define processing chain
            chain = (
                {"context": retriever, "questions": RunnablePassthrough()}
                | prompt
                | model
                | StrOutputParser()
            )
            
            # Invoke the processing chain with user input
            response = chain.invoke(query)
        
        else:
            print('The Data source format is not in required format')

        return response

            

In [24]:
chat = DataTalk(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\document_0.docx')
chat.generate_response('What is the document about?')

100%|██████████| 1/1 [00:04<00:00,  4.32s/it]


'The document is about artist requirements for an event in Abu Dhabi by UltraTech Cement. It includes emails with artist options, costs, and availability for the event.'

In [6]:
import os
import sys
import requests
from io import BytesIO
import fitz  # PyMuPDF
from langchain.document_loaders import CSVLoader, PyMuPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from docx2pdf import convert
import pandas as pd
from dotenv import load_dotenv
import openai
from PyPDF2 import PdfReader

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [7]:
class DataTalk:

    def __init__(self, cdn_url):
        self.cdn_url = cdn_url
        self.filepath = self.download_file(cdn_url)

    def download_file(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            cdn_filename = url.split('/')[-1]
            with open(cdn_filename, 'wb') as f:
                f.write(response.content)
            print(f"File {cdn_filename} downloaded successfully.")
            return cdn_filename
        else:
            raise Exception(f"Failed to download file from {url}")

    def source_type(self):
        file = ''
        folderpath = os.path.dirname(self.filepath)
        filename = os.path.basename(self.filepath)
        if filename.endswith('.xlsx'):
            csvname = filename.split('.')[0] + '.csv'
            file = os.path.join(folderpath, csvname)
            df = pd.read_excel(self.filepath)
            df.to_csv(file, index=False)
        elif filename.endswith('.docx'):
            pdfname = filename.split('.')[0] + '.pdf'
            file = os.path.join(folderpath, pdfname)
            convert(self.filepath, file)
        else:
            file = self.filepath

        return file

    def generate_response(self, query):
        embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
        model_name = "gpt-3.5-turbo-instruct"
        file = self.source_type()
        response = ''

        if file.endswith('.csv'):
            loader = CSVLoader(file_path=file)
            index_creator = VectorstoreIndexCreator(embedding=embeddings)
            docsearch = index_creator.from_loaders([loader])
            llm = OpenAI(model_name=model_name, openai_api_key=openai.api_key)
            chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
            response = chain({"question": query})
            response = response['result']

        elif file.lower().endswith('.pdf'):
            pdf_reader = PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
            chunks = text_splitter.split_text(text=text)

            vector_store = FAISS.from_texts(chunks, embedding=embeddings)
            retriever = vector_store.as_retriever()

            template = """Respond to the prompt based on the following context: {context}
            Questions: {questions}
            """
            prompt = ChatPromptTemplate.from_template(template)
            model = ChatOpenAI(openai_api_key=openai.api_key)

            chain = (
                {"context": retriever, "questions": RunnablePassthrough()}
                | prompt
                | model
                | StrOutputParser()
            )

            response = chain.invoke(query)

        else:
            print('The data source format is not in the required format.')

        return response


In [8]:
chat = DataTalk('https://github.com/py-pdf/sample-files/blob/main/001-trivial/minimal-document.pdf')
chat.generate_response('What is the doc about?')

File minimal-document.pdf downloaded successfully.


PdfReadError: EOF marker not found

In [9]:
import os
import requests
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
import PyPDF2
from PyPDF2 import PdfReader
from docx2pdf import convert
import pandas as pd
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [12]:
class DataTalk:

    def __init__(self, url):
        self.url = url
        self.filepath = self.download_file(url)

    def download_file(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            # Extract the filename from the URL and save it in the current directory
            filename = url.split("/")[-1].split("?")[0]  # Remove query parameters if any
            with open(filename, "wb") as file:
                file.write(response.content)
            return filename
        else:
            raise Exception(f"Failed to download file from {url}")

    def source_type(self):
        file = ''
        folderpath = os.path.dirname(self.filepath)
        filename = os.path.basename(self.filepath)
        if filename.endswith('.xlsx'):
            csvname = filename.split('.')[0] + '.csv'
            file = os.path.join(folderpath, csvname)
            df = pd.read_excel(self.filepath)
            df.to_csv(file)
        elif filename.endswith('.docx'):
            pdfname = filename.split('.')[0] + '.pdf'
            file = os.path.join(folderpath, pdfname)
            convert(self.filepath, file)
        else: 
            file = self.filepath

        return file

    def generate_response(self, query):
        # Initialize the OpenAI embeddings
        embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
        model_name = "gpt-3.5-turbo-instruct"
        file = self.source_type()
        response = ''

        if file.endswith('.csv'):
            # Load the documents
            loader = CSVLoader(file_path=file)
            # Create an index using the loaded documents
            index_creator = VectorstoreIndexCreator(embedding=embeddings)
            docsearch = index_creator.from_loaders([loader])
            # Create a question-answering chain using the index
            llm = OpenAI(model_name=model_name, openai_api_key=openai.api_key)
            chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
            response = chain({"question": query})
            response = response['result']

        elif file.lower().endswith('.pdf'):
            """
            Process a PDF file, extracting text and splitting it into chunks.
            """
            pdf_reader = PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            
            # Split text into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            chunks = text_splitter.split_text(text=text)

            # Create vector store using FAISS
            vector_store = FAISS.from_texts(chunks, embedding=embeddings)

            """
            Generate responses based on prompts and embeddings.
            """
            retriever = vector_store.as_retriever()
            
            # Define template for prompts
            template = """Respond to the prompt based on the following context: {context}
            Questions: {questions}
            """
            prompt = ChatPromptTemplate.from_template(template)

            # Initialize ChatOpenAI model
            model = ChatOpenAI(openai_api_key=openai.api_key)
            
            # Define processing chain
            chain = (
                {"context": retriever, "questions": RunnablePassthrough()}
                | prompt
                | model
                | StrOutputParser()
            )
            
            # Invoke the processing chain with user input
            response = chain.invoke(query)
        
        else:
            print('The data source format is not in required format')

        return response

# Example usage
chat = DataTalk('https://drive.google.com/uc?id=1zO8ekHWx9U7mrbx_0Hoxxu6od7uxJqWw&export=download')
response = chat.generate_response('What is the doc about?')
print(response)

The data source format is not in required format

